In [24]:
import pandas as pd
import numpy as np

In [2]:
# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject=CS" # get SP 22 CS classes
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())


In [3]:
classes = data_json['data']['classes']

In [25]:
# CS 1110, Bracy, start time, end time, location, description, prereqs/coreqs, semester offered
df = []
missedClasses = 0

for subject in ["CS", "INFO", "ECE", "ECON", "ENGL", "PHIL", "AAS", "AEM", "AEP", "AGSCI", "AIIS", "AIRS",
               "AKKAD", "ALS", "AMST", "ANSC", "ANTHR", "BTRY", "BIOAP", "BIOEE", "BIOG", "BIOMG", "BIOMI",
               "BIOMS", "BIONB", "BME"]: #, "ECON"]"ENGL", "PHIL"]:

    url = f"https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject={subject}"
    response = urlopen(url)

    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())

    classes = data_json['data']['classes']

    for class_ in classes:
        #print(class_)
        try:
            #if (int(class_['catalogNbr'])) > 5900:
            #    continue
            #print(class_)
            obj = class_['enrollGroups'][0]['classSections'][0]['meetings'][0]

            obj2 = obj['instructors'][0]
            #print(obj)

            startTime = obj['timeStart']
            endTime = obj['timeEnd']
            location = obj['facilityDescr']
            x = [class_['subject'] , class_['catalogNbr'] , class_['titleLong'], obj2['firstName'].strip() + ' ' + obj2['lastName'].strip(), startTime, endTime, location, class_['description'], class_['catalogPrereqCoreq'], class_['catalogWhenOffered']]
            df.append(x)
        except:
            missedClasses += 1
print(missedClasses)

41


In [26]:
df1 = pd.DataFrame(np.array(df))
df1.shape

(918, 10)

In [27]:
df1 = df1.rename(columns={0:"Dept", 1:"Number", 2:"Course_Name", 3:"Professor", 4:"Start_Time", 5:"End_Time", 6:"Location",7:"Description",8:"Pre/coreqs",9:"Offered"})

In [33]:
median_grades = pd.read_csv("Cornell_Median_Grades.csv")

,Dept + Number,Professor,Median Grade,Semester (Ex: SP21),# of Students,Notes
0,SEARCH BEFORE ADDING YOUR CLASS,NaN,NaN,NaN,NaN,NaN
1,AAS 2130,Chang,A,FA21,52,"crosslisted with AMST 2640, HIST 2640"
2,AAS 2130,Moghimi,A,FA21,NaN,NaN
3,AAS 2130,van Es,B+,FA21,239,NaN
4,AEM 2010,Haeger,A-,FA21,138,NaN
...,...,...,...,...,...,...
630,VIEN 1104,Arnink/Moss,A,FA21,208,NaN
631,NaN,Hazan,A,SP21,706,NaN
632,NaN,Poitras,B,FA21,588,NaN
633,NaN,van Zuylen,B+,SP21,315,NaN


In [34]:
median_grades = median_grades[['Dept + Number', 'Median Grade', 'Semester (Ex: SP21)']]

In [35]:
df1["Dept + Number"] = df1["Dept"] + ' ' + df1["Number"]

In [36]:
df2 = df1.merge(median_grades, on='Dept + Number', how="outer")

In [37]:
df2 = df2.dropna(subset=['Dept'])

In [38]:
df2

,Dept,Number,Course_Name,Professor,Start_Time,End_Time,Location,Description,Pre/coreqs,Offered,Dept + Number,Median Grade,Semester (Ex: SP21)
0,CS,1110,Introduction to Computing Using Python,Anne Bracy,09:05AM,09:55AM,Bailey Hall 101,Programming and problem solving using Python. ...,,"Fall, Spring, Summer.",CS 1110,B+,FA21
1,CS,1112,Introduction to Computing Using MATLAB,Kit-Yee Daisy Fan,02:40PM,03:30PM,Hollister Hall B14,Programming and problem solving using MATLAB. ...,"Corequisite: MATH 1110, MATH 1910, or equivalent.","Fall, Spring.",CS 1112,B,FA21
2,CS,1132,Short Course in MATLAB,Kit-Yee Daisy Fan,02:40PM,03:30PM,Phillips Hall 219,Introduction to the MATLAB programming languag...,,"Fall, Spring.",CS 1132,NaN,NaN
3,CS,1133,Short Course in Python,Craig Frey,12:25PM,01:15PM,Upson Hall 142,Introduction to the Python programming languag...,,"Fall, Spring.",CS 1133,NaN,NaN
4,CS,1300,Introductory Design and Programming for the Web,Royal Westwater,11:25AM,12:40PM,Olin Hall 165,The World Wide Web is both a technology and a ...,,"Fall, Spring.",CS 1300,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,BME,6410,Cell and Molecular Mechanobiology,Jan Lammerding,09:40AM,10:55AM,Weill Hall 224,Mechanobiology describes how cells and tissues...,"Prerequisite: BIOMG 1350 or equivalent, BME 30...",Spring.,BME 6410,NaN,NaN
937,BME,7020,Biomedical Engineering Research Seminar,Peter Doerschuk,09:05AM,09:55AM,Weill Hall 226,Designed to train graduate students in public ...,,"Fall, Spring.",BME 7020,NaN,NaN
938,BME,7030,Graduate Student Teaching Experience,Jan Lammerding,,,To Be Assigned,Guided individual experience in laboratory ins...,,"Fall, Spring.",BME 7030,NaN,NaN
939,BME,7130,Core Concepts in Disease,Nate Cira,11:25AM,12:40PM,Weill Hall 226,Most diseases emerge due to a relative small n...,Prerequisite: introductory biology.,Spring.,BME 7130,NaN,NaN


In [40]:
df2.to_csv("Classes_With_Medians.csv")